# New project name
> Explain what this notebook does specifically and link to the data source. 

---

#### Import Python tools and Jupyter config

In [18]:
import requests
import pandas as pd
from tqdm import tqdm
from tabulate import tabulate

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
}

country_list = []

# Initialize parameters for the first request
params = {
    "filter[inactive]": "0",
    "page[offset]": "0",
    "page[limit]": "50",
}

# Loop until there are no more pages
while True:
    # Make the request
    response = requests.get(
        "https://www.nbcolympics.com/api/country", params=params, headers=headers
    )

    # Check if the request was successful
    if response.status_code != 200:
        print("Request failed with status code:", response.status_code)
        break

    data = response.json()

    # Extract country data from the response
    for c in data["data"]:
        attributes = c["attributes"]
        country_list.append(
            {
                "id": attributes["gracenote_id"],
                "name": attributes["name"],
                "code": attributes["code"],
                "slug": attributes["url_id"],
            }
        )

    # Check if there's a next page
    next_link = data.get("links", {}).get("next", {}).get("href")
    if not next_link:
        break

    # Update the params for the next request
    # Extract the new offset from the next link, if it exists
    params["page[offset]"] = next_link.split("page%5Boffset%5D=")[1].split("&")[0]

# Display the collected country data
countries = pd.DataFrame(country_list)

nocIds = countries.id.to_list()

print("---")
print(f'Found {len(nocIds)} participating countries...')

# List to store the results
medal_results = []

# Loop over each NOC ID to get the medal details
print("---")
print(f'Fetching medal counts from {len(nocIds)} countries...')

for nocId in tqdm(nocIds):
    params = {
        "competitionSetId": "1",
        "season": "2024",
        "nocId": str(nocId),
        "languageCode": "2",
    }

    results_response = requests.get(
        "https://api-gracenote.nbcolympics.com/svc/games_v2.svc/json/GetMedalTableNOCDetail_Season",
        params=params,
        headers=headers,
    )

    # Check if the request was successful
    if results_response.status_code != 200:
        print(
            f"Request failed for NOC ID {nocId} with status code: {results_response.status_code}"
        )
        continue

    # Get the JSON data
    data = results_response.json()

    # Check if 'NOCMedals' is present and not None
    json_data = data.get("NOCMedals", None)

    if json_data is None:
        # Handle countries with no medals
        results_dict = {
            "country": None,
            "gold": 0,
            "silver": 0,
            "bronze": 0,
            "total": 0,
        }
    else:
        # Extract data when medals are available
        noc_info = json_data.get("NOC", {})
        medals_info = json_data.get("Medals", {})

        results_dict = {
            "country": noc_info.get("c_Name", None),
            "gold": medals_info.get("n_Gold", 0),
            "silver": medals_info.get("n_Silver", 0),
            "bronze": medals_info.get("n_Bronze", 0),
            "total": medals_info.get("n_Total", 0),
        }

    # Append the results to the list
    medal_results.append(results_dict)

df = (
    pd.DataFrame(medal_results)
    .sort_values(["gold", "total"], ascending=False)
    .reset_index(drop=True)
)

winners = df.query('total>0')
winner_count = len(winners)

print("---")
print(f"Found {winner_count} countries that won at least one medal.")

winner = df[['country', 'gold', 'silver', 'bronze', 'total']]['country'].iloc[0]

print("---")
print(f"{winner} tops the list! ")

table = df[['country', 'gold', 'silver', 'bronze', 'total']].head(10)
print(tabulate(table, headers='keys', tablefmt='mixed_grid', showindex=False))

---
Found 202 participating countries...
---
Fetching medal counts from 202 countries...


100%|██████████████████████████████████████████████████████████████████████| 202/202 [01:38<00:00,  2.06it/s]

---
Found 90 countries that won at least one medal.
---
United States tops the list! 
+---------------+--------+----------+----------+---------+
| country       |   gold |   silver |   bronze |   total |
|---------------+--------+----------+----------+---------|
| United States |     40 |       44 |       42 |     126 |
| China         |     40 |       27 |       24 |      91 |
| Japan         |     20 |       12 |       13 |      45 |
| Australia     |     18 |       19 |       16 |      53 |
| France        |     16 |       26 |       22 |      64 |
| Netherlands   |     15 |        7 |       12 |      34 |
| Great Britain |     14 |       22 |       29 |      65 |
| South Korea   |     13 |        9 |       10 |      32 |
| Italy         |     12 |       13 |       15 |      40 |
| Germany       |     12 |       13 |        8 |      33 |
+---------------+--------+----------+----------+---------+


┍━━━━━━━━━━━━━━━┯━━━━━━━━┯━━━━━━━━━━┯━━━━━━━━━━┯━━━━━━━━━┑
│ country       │   gold │   silver │   bronze │   total │
┝━━━━━━━━━━━━━━━┿━━━━━━━━┿━━━━━━━━━━┿━━━━━━━━━━┿━━━━━━━━━┥
│ United States │     40 │       44 │       42 │     126 │
├───────────────┼────────┼──────────┼──────────┼─────────┤
│ China         │     40 │       27 │       24 │      91 │
├───────────────┼────────┼──────────┼──────────┼─────────┤
│ Japan         │     20 │       12 │       13 │      45 │
├───────────────┼────────┼──────────┼──────────┼─────────┤
│ Australia     │     18 │       19 │       16 │      53 │
├───────────────┼────────┼──────────┼──────────┼─────────┤
│ France        │     16 │       26 │       22 │      64 │
├───────────────┼────────┼──────────┼──────────┼─────────┤
│ Netherlands   │     15 │        7 │       12 │      34 │
├───────────────┼────────┼──────────┼──────────┼─────────┤
│ Great Britain │     14 │       22 │       29 │      65 │
├───────────────┼────────┼──────────┼──────────┼────────